In [141]:

import os
import pandas as pd
import urllib
import requests
import math
import json
import gspread
from pathlib import Path
from datetime import datetime
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain import hub
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.output_parsers import RetryOutputParser
from dotenv import load_dotenv
from langchain_community.chat_models import ChatPerplexity


load_dotenv()

global nps_advisor_evaluation_folder
nps_advisor_evaluation_folder = '1iMwf0suglb9XnovTsVGRgE25mSCERW2p' # Output folder for AI evaluation

DOC_ID = '1ifHt5uJJ4uUeaF2O_qOJXmfLR-LCF64w8ZUopWJ2xmQ' # ID for google sheet script will write to

# Authentication for gsheets
gc = gspread.oauth(
    credentials_filename='credentials.json',
    authorized_user_filename='token.json'
)

# Load test cases and data store
ss = gc.open_by_key(DOC_ID)
worksheet = ss.worksheet("reference_scores")
testcases_df = pd.DataFrame(worksheet.get_all_records())

reference_scores_gsheet = gc.open_by_key(DOC_ID)
reference_scores_sheet = reference_scores_gsheet.worksheet("reference_scores")
reference_scores_df = pd.DataFrame(reference_scores_sheet.get_all_records())

# Define variables
relevance = "relevance"
depth = "depth"
prioritization = "prioritization"

relevance_reference_header = "nps_advisor_relevance"
depth_reference_header = "nps_advisor_depth"
prioritization_reference_header = "nps_advisor_priority"

url = 'https://agenticworkflows.onrender.com/invoke/'
reframe = 'true'
guardrails = 'true'
telco_operator = 'Circles.Life'
payload_params = [url, reframe, guardrails, telco_operator]
output_dir = '/output/'

# Pull latest prompt from LangSmith
relevance_prompt = hub.pull("benchmarking_relevance_v2")
depth_prompt = hub.pull("benchmarking_depth_v2")
priority_prompt = hub.pull("benchmarking_prioritization_v1")

prompts = {'relevance': relevance_prompt,
           'depth': depth_prompt,
           'prioritization': priority_prompt}

# Initialize models to test
gpt4_model = {'name': 'gpt', 'model': ChatOpenAI(
    model="gpt-4o", temperature=0.5)}
gemini_model = {'name': 'gemini',
                'model': ChatGoogleGenerativeAI(model="gemini-1.5-flash")}
nps_advisor = {'name': 'nps_advisor'}
perplexity_model = {'name': 'perplexity', 'model': ChatPerplexity(temperature=0, model="llama-3.1-sonar-small-128k-online")}
reference_model = {'name': 'reference'}

test_models = ['nps_advisor', 'gpt', 'gemini', 'perplexity', 'reference']

# Initialize evaluator model
eval_model = ChatOpenAI(model="gpt-4o", temperature=0.0)
# eval_model = ChatGoogleGenerativeAI(model="gemini-pro")


# Define evaluation function
def evaluate_criteria_from_file(parser, eval_llm, prompt, testcases_df):
    eval_store_data = []

    # Iterate over test cases and evaluate answers from file
    for index, row in testcases_df.iterrows():
        question = row["question"]
        reference = row["reference"]
        answer = row["nps_advisor_answer"]

        # Evaluate response
        eval_prompt_and_model = prompt | eval_llm
        output = eval_prompt_and_model.invoke(
            {"question": question, "answer": answer, "reference": reference})

        # Parse the output using the parser
        parsed_result = parser.invoke(output)

        # Store results
        eval_store_data.append(parsed_result)
        eval_store_data[index]["question"] = question
        eval_store_data[index]["reference"] = reference
        eval_store_data[index]["answer"] = answer
    return eval_store_data

# Define evaluation function
def evaluate_llm_output(parser, eval_llm, prompt, testcases_df, answers):
    eval_store_data = []

    # Evaluate responses
    for index, row in testcases_df.iterrows():
        question = testcases_df.loc[index]['question']
        reference = testcases_df.loc[index]['reference']

        eval_prompt_and_model = prompt | eval_llm
        output = eval_prompt_and_model.invoke(
            {"question": question, "answer": answers[index], "reference": reference})

        # Parse the output using the parser
        parsed_result = parser.invoke(output)

        # Store results
        eval_store_data.append(parsed_result)
        eval_store_data[index]["question"] = question
        eval_store_data[index]["reference"] = reference
        eval_store_data[index]["answer"] = answers[index][1]

    return eval_store_data

# Create a prompt
def create_prompt(prompt_template, criteria):
    # Define output schema
    response_schemas = [
        ResponseSchema(name="evaluation", description="feedback on answer"),
        ResponseSchema(
            name=criteria,
            description="evaluation of answer, must be a percentage",
        ),
    ]

    # Define pydanthic output parser
    output_parser = StructuredOutputParser.from_response_schemas(
        response_schemas)
    format_instructions = output_parser.get_format_instructions()

    eval_prompt = PromptTemplate(
        template=prompt_template.template,
        input_variables=["question", "answer", "reference"],
        partial_variables={"format_instructions": format_instructions},
    )
    return eval_prompt, output_parser

# Format score into float
def format_score(criteria, dataframe):
    dataframe[criteria] = dataframe[criteria].str.rstrip(
        '%').astype('float')/100.0
    return dataframe


def calculate_score(relevance, depth, prioritization):
    if math.isnan(prioritization):
        return relevance * 0.500 + depth * 0.500
    return relevance * 0.450 + depth * 0.450 + prioritization * 0.100


def apply_calculate_score(dataframe):
    return calculate_score(dataframe['relevance'], dataframe['depth'], dataframe['prioritization'])


def get_answers(testcases_df, tested_llm, payload_params):
    answers = []

    if tested_llm['name'] == 'nps_advisor':
        url = payload_params[0]
        reframe = payload_params[1]
        guardrails = payload_params[2]
        telco_operator = payload_params[3]

        # Loop through question list and get responses
        for index, row in testcases_df.iterrows():
            question = row["question"]
            question_reframed = urllib.parse.quote_plus(question)

            # # Get answer from LLM
            payload = "/".join([question_reframed, reframe,
                                guardrails, telco_operator])
            answer = requests.get(url + payload)
            answers.append((index, json.loads(answer.content)['response'][0]))

    else:
        for index, row in testcases_df.iterrows():
            question = row["question"]

            # # Get answer from LLM
            answer = tested_llm['model'].invoke(
                [HumanMessage(content=question)]).content
            answers.append((index, answer))

    return answers


def evaluation(prompt, criteria, testcases, test_target, answers):
    test_models = ['nps_advisor', 'gpt', 'gemini']
    headers = ["question", "reference", "answer", "evaluation"]
    headers.append(criteria)

    prompt, parser = create_prompt(prompt, criteria)
    if criteria == "prioritization":  # Select only rows with prioritization flag
        testcases = testcases.loc[testcases['prioritization_flag'] == 1]

    if test_target['name'] == 'reference':
        store_data = evaluate_criteria_from_file(
            parser, eval_model, prompt, testcases)
        df = pd.DataFrame(store_data, columns=headers)
        df = format_score(criteria, df)
        
        Path(os.getcwd() + output_dir + test_target['name'] + "/").mkdir(parents=True, exist_ok=True)
        df.to_excel(os.getcwd() + 
                    output_dir + 
                    test_target['name'] + "/" +
                    datetime.today().strftime('%Y%m%d') + "_" + criteria + "_score.xlsx")
    else:
        store_data = evaluate_llm_output(
            parser, eval_model, prompt, testcases, answers)
        
        df = pd.DataFrame(store_data, columns=headers)
        df = format_score(criteria, df)

        sh = gc.create(datetime.today().strftime('%Y%m%d') + "_" +
                    criteria + "_scoring.xlsx",
                    folder_id=nps_advisor_evaluation_folder)
        worksheet = sh.get_worksheet(0)
        worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    return df


def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string


# Main exec
def execute_benchmarks(checks, prompts, testcases, test_target, answers):
    for check in checks:
        match check:
            case "relevance":
                criteria = 'relevance'
                prompt = prompts[criteria]
                rel_df = evaluation(
                    prompt, criteria, testcases, test_target, answers)
            case "depth":
                criteria = 'depth'
                prompt = prompts[criteria]
                dep_df = evaluation(
                    prompt, criteria, testcases, test_target, answers)
            case "prioritization":
                criteria = 'prioritization'
                prompt = prompts[criteria]
                pri_df = evaluation(
                    prompt, criteria, testcases, test_target, answers)

    # Calculate weighted score
    if len(checks) == 3:
        dep_join = dep_df.columns.difference(rel_df.columns)
        pri_join = pri_df.columns.difference(rel_df.columns)
        output_df = pd.merge(rel_df, dep_df[dep_join], left_index=True,
                             right_index=True, how="outer")
        output_df = pd.merge(output_df, pri_df[pri_join], left_index=True,
                             right_index=True, how="outer")
        output_df["weighted_score"] = output_df.apply(
            apply_calculate_score, axis=1)

        return output_df
    else:
        return ''


##### Execution Variables ######
# Target to test: gpt4_model, gemini_model, nps_advisor, perplexity_model, reference_model
test_target = nps_advisor

# Switches for debugging
checks = ['relevance', 'depth', 'prioritization']

# Main function
def main():
    if test_target['name'] == 'reference':
        answers = ''
    else:
        answers = get_answers(testcases_df, test_target, payload_params)

    weighted_score_df = execute_benchmarks(checks,
                                           prompts,
                                           testcases_df,
                                           test_target,
                                           answers)

    if test_target['name'] == 'nps_advisor':
        weighted_today = weighted_score_df.loc[:, 'weighted_score'].astype(str)
        ss = gc.open_by_key(DOC_ID)
        worksheet = ss.worksheet("benchmarking")
        values = worksheet.get_all_values()
        col = colnum_string(max([len(r) for r in values]) + 1)
        worksheet.update(col + '1', [[datetime.today().strftime('%Y%m%d')]] + [[e]
                                                                               for e in weighted_today.tolist()], value_input_option='USER_ENTERED')
    
    if test_target['name'] == 'reference':
        Path(os.getcwd() + output_dir).mkdir(parents=True, exist_ok=True)
        weighted_score_df.to_excel(os.getcwd() + output_dir +
                                   datetime.today().strftime('%Y%m%d') + "_" +
                                   test_target['name'] +
                                   "_weighted_score.xlsx")

main()

/var/folders/yr/xqb6wgvd2mx6wkb_clsz33fh0000gq/T/ipykernel_605/4270990534.py:311: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update(col + '1', [[datetime.today().strftime('%Y%m%d')]] + [[e]


In [137]:
import pandas as pd
import math
import os


def calculate_score(relevance, depth, prioritization):
    if math.isnan(prioritization):
        return relevance * 0.500 + depth * 0.500
    return relevance * 0.450 + depth * 0.450 + prioritization * 0.100


def apply_calculate_score(dataframe):
    return calculate_score(dataframe['relevance'], dataframe['depth'], dataframe['prioritization'])


dep_join = dep_df.columns.difference(rel_df.columns)
pri_join = pri_df.columns.difference(rel_df.columns)
output_df = pd.merge(rel_df, dep_df[dep_join], left_index=True,
                     right_index=True, how="outer")
output_df = pd.merge(output_df, pri_df[pri_join], left_index=True,
                     right_index=True, how="outer")

# display(output_df)
output_df["weighted_score"] = output_df.apply(apply_calculate_score, axis=1)

Path(os.getcwd() + output_dir).mkdir(parents=True, exist_ok=True)
output_df.to_excel(os.getcwd() + output_dir +
                   test_target['name'] + "_weighted_score.xlsx")

In [ ]:
display(weighted_score_df.loc[:, 'weighted_score'])
weighted_today = weighted_score_df.loc[:, 'weighted_score']
weighted_today.values.to_list()

In [103]:
import gspread
import pandas as pd

# weighted_today = weighted_score_df.loc[:, 'weighted_score'].astype(str)
# weighted_today.describe


def colnum_string(n):
    string = ""
    while n > 0:
        n, remainder = divmod(n - 1, 26)
        string = chr(65 + remainder) + string
    return string


gc = gspread.oauth(
    credentials_filename='/Users/jenyang.niam/Projects/benchmarking_automation/credentials.json',
    authorized_user_filename='/Users/jenyang.niam/Projects/benchmarking_automation/token.json'
)
# gc = gspread.oauth()

# gc = gspread.service_account(filename='credentials.json')
DOC_ID = '1ifHt5uJJ4uUeaF2O_qOJXmfLR-LCF64w8ZUopWJ2xmQ'
SHEET_NAME = 'Sheet1'
DOC_URL = 'https://docs.google.com/spreadsheets/d/' + \
    DOC_ID + '/gviz/tq?tqx=out:csv&sheet=' + SHEET_NAME

ss = gc.open_by_key(DOC_ID)
worksheet = ss.worksheet("reference_scores")


testcases_df = pd.DataFrame(worksheet.get_all_records())

sh = gc.create('A new spreadsheet',
               folder_id='1NirPog7MubSuwe4Jv4dspU6EzHV9XTJL')
worksheet2 = sh.get_worksheet(0)
worksheet2.update([testcases_df.columns.values.tolist()] +
                  testcases_df.values.tolist())
# for index, row in testcases_df.iterrows():
#     print(index, row)

# testcases_df = pd.read_csv("testcases_v1.csv")
# print(questions)
# print(reference)
# rang = 'A' + str(len(values_list)+1)
# worksheet.update(rang, [["20240911"]] + [[e] for e in weighted_today.tolist()])


# values = worksheet.get_all_values()
# col = colnum_string(max([len(r) for r in values]) + 1)
# worksheet.update(col + '1', [["20240911"]] + [[e]
#                  for e in weighted_today.tolist()], value_input_option='USER_ENTERED')

{'spreadsheetId': '1Yb83yEk36ptZa0dzg_i4jAr8koe43MXNkJVd66431To',
 'updatedRange': 'Sheet1!A1:Q16',
 'updatedRows': 16,
 'updatedColumns': 17,
 'updatedCells': 272}